In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional,LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder

# Load data from CSV file
data = pd.read_csv('/content/augmented_data (1) - Copy.csv')

# Preprocess data
X = data['Questions'].values
y = data['Blooms Taxonomy'].values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert input text to BERT token IDs and pad sequences
max_len = 128
X_encoded = tokenizer.batch_encode_plus(X, add_special_tokens=True, max_length=max_len, padding='max_length', truncation=True)
X_padded = pad_sequences(X_encoded['input_ids'], maxlen=max_len, dtype='long', truncating='post', padding='post')

# Split data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X_padded[:train_size], X_padded[train_size:]
y_train, y_test = y_encoded[:train_size], y_encoded[train_size:]

# Define a custom learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.96)

# Build and train model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.vocab), output_dim=100, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 100)          3052200   
                                                                 
 bidirectional (Bidirection  (None, 128, 128)          84480     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 128, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 38)                1

In [2]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=16)

Epoch 1/30
153/153 [==============================] - 30s 137ms/step - loss: 2.4904 - accuracy: 0.2446 - val_loss: 1.8711 - val_accuracy: 0.3235
Epoch 2/30
153/153 [==============================] - 8s 55ms/step - loss: 1.5800 - accuracy: 0.5351 - val_loss: 1.5177 - val_accuracy: 0.5369
Epoch 3/30
153/153 [==============================] - 5s 35ms/step - loss: 1.0620 - accuracy: 0.6938 - val_loss: 1.5115 - val_accuracy: 0.5747
Epoch 4/30
153/153 [==============================] - 5s 33ms/step - loss: 0.7696 - accuracy: 0.7949 - val_loss: 1.4117 - val_accuracy: 0.6059
Epoch 5/30
153/153 [==============================] - 5s 33ms/step - loss: 0.6175 - accuracy: 0.8220 - val_loss: 1.4835 - val_accuracy: 0.5928
Epoch 6/30
153/153 [==============================] - 5s 30ms/step - loss: 0.4774 - accuracy: 0.8640 - val_loss: 1.7815 - val_accuracy: 0.5928
Epoch 7/30
153/153 [==============================] - 5s 33ms/step - loss: 0.3918 - accuracy: 0.8952 - val_loss: 1.8020 - val_accuracy: 0.61

In [3]:
model.evaluate(X_test,y_test)

20/20 [==============================] - 0s 8ms/step - loss: 2.2508 - accuracy: 0.6732


[2.2508199214935303, 0.6732348203659058]